In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
from genpeds import Admissions, Enrollment, Completion, Graduation
from cfg import THEME

pio.templates['THEME'] = THEME
pio.templates.default='THEME'

## **Use NCES IPEDS data with `genpeds`**

### **Admissions**

#### Load in 2023 admissions data

In [2]:
adm = Admissions(year_range=[2022,2023]).run(see_progress=False, # DO NOT print download progress statements
                                          merge_with_char=True, # MERGE with characteristics (e.g. address, zipcode)
                                          rm_disk=True) # RM from storage after cleaning

#### go.geoscatter

In [3]:
pio.renderers.default = 'browser'


adm = adm.dropna(subset=['tot_enrolled', 'act_comp_50'])
adm2023 = adm.query('act_comp_50 > 5').copy()

fig_text = (
    #'<span style="font-family: Georgia, serif; text-transform: none;">' + 
    '<b>' + adm2023['name'] + '</b>' +
    '<br>' + adm2023['city'] + ', ' + adm2023['state']  + 
    '<br><b>Median ACT Composite</b>: ' + adm2023['act_comp_50'].astype(str) 
)

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode='USA-states',
    lon=adm2023['longitude'],
    lat=adm2023['latitude'],
    text=fig_text,
    marker={
        'color': adm2023['act_comp_50'],
        'size' : np.log(adm2023['tot_enrolled']) * 2, 
        'sizemode': 'diameter'
    }
    
))

fig.update_layout(
    title_text=#'<span style="font-family: Georgia, serif; font-size: 24px; color: #1E4A4A; text-transform: none; font-weight: bold;">' \
    'Northeastern Schools have higher ACT scores</span>',
    title={'subtitle': {'text': 'Median ACT Composite by School (2023)'}},
    showlegend=True,
    # geo={
    #     'scope': 'usa',
    #     'bgcolor': '#ffffff',
    #     'landcolor': '#ffffff',
    #     'subunitcolor': '#999999'
    # }
)
fig.show()